In [10]:
# from fastai import __version__
# print(__version__)

2.5.2


In [1]:
# !jupyter serverextension enable --sys-prefix voila

Enabling: voila
- Writing config: C:\Users\SHARAD\anaconda3\etc\jupyter
    - Validating...
      voila 0.2.16 ok


In [2]:
#from fastbook import *

In [3]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [3]:
# from ipywidgets import *

In [5]:
# import torch
# import fastai

In [5]:
# import pathlib
# temp = pathlib.PosixPath
# pathlib.PosixPath = pathlib.WindowsPath

In [6]:
# fix failure to preserve metadata in multiprocessing in DataLoader
def _rebuild_from_type(func, type, args, dict):
    ret = func(*args).as_subclass(type)
    ret.__dict__ = dict
    return ret

@patch
def __reduce_ex__(self: TensorBase, proto):
    from fastai.torch_core import _fa_rebuild_qtensor, _fa_rebuild_tensor
    torch.utils.hooks.warn_if_has_hooks(self)
    args = (type(self), self.storage(), self.storage_offset(), tuple(self.size()), self.stride())
    if self.is_quantized: args = args + (self.q_scale(), self.q_zero_point())
    args = args + (self.requires_grad, OrderedDict())
    f = _fa_rebuild_qtensor if self.is_quantized else  _fa_rebuild_tensor
    return (_rebuild_from_type, (f, type(self), args, self.__dict__))

In [7]:
learn_inf = load_learner('export.pkl', cpu=True)

In [8]:
def on_data_change2(change):
    lbl_pred.value = ''
    img1 = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img1.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img1)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [12]:
xx =widgets.HTML(
    value="""
    <h1><b>Mask Detector</b></h1>
    <p>Detect whether a person is wearing a mask or not. It will <b>not</b> give sensible results for pictures without a human face.</p>
    """
)
btn_upload = widgets.FileUpload(accept='.jpg,.jpeg',multiple=False)
out_pl = widgets.Output()
lbl_pred = widgets.Label()
lbl_pred2 = widgets.Label()
lbl_pred2.value=str(learn_inf.dls.vocab)
btn_upload.observe(on_data_change2, names=['data'])
display(VBox([xx,widgets.Label('Upload the Photo!'), btn_upload, lbl_pred,lbl_pred2,out_pl ]))